In [1]:
import sys
sys.path.append("C://Users//User//PycharmProjects//Musgrave_scripts//BenRich//common_funcs")
import cohpfuncs as cfunc
import jdftxfuncs as jfunc
data_dir = "E://perl//pscratch//beri9208//pot_scan//PtPyr//"
import numpy as np
import matplotlib.pyplot as plt
save_dir = "C://Users//User//Desktop//pcoop_plots//"
import time
import os

In [2]:
def app_dir(surf_idx, mu_idx):
    """
    :param surf_idx: 0 --> 2n-cis, 1 --> 2n-trans
    :param mu_idx: 0 --> No_bias, -1 --> dneg/1, 1 --> 2pos/1
    :return:
    """
    out_str = ""
    if surf_idx:
        out_str += "2n-trans//"
    else:
        out_str += "2n-cis//"
    if mu_idx == 0:
        out_str += "No_bias//"
    else:
        if mu_idx > 0:
            out_str += "dpos//"
        else:
            out_str += "dneg//"
        out_str += str(int(abs(mu_idx))) + "//"
    return data_dir + out_str

def lb_idx(Emin, dE, sample):
    a = sample-Emin
    b = a/dE
    c = int(np.floor(b))
    return c, abs(b-c)

def lsstr(atom_label):
    saveable_string = ""
    use = atom_label.split(" #")
    for u in use:
        saveable_string += u
    return saveable_string

surf_labels = ["cis", "trans"]
bias_labels = ["-0.3V", "0.0V", "0.3V", "0.6", "0.9V", "-1.5V", "-1.2V","-0.9V", "-0.6V"]

def gen_dirname(lookat1, lookat2, surf_idx, save_dir):
    slabel = surf_labels[surf_idx]
    out_str = save_dir + slabel
    out_str += lookat1.split(' #')[0]
    out_str += lookat1.split(' #')[1]
    out_str += '-'
    out_str += lookat2.split(' #')[0]
    out_str += lookat2.split(' #')[1]
    out_str += '//'
    return out_str

def smear(x, center, wid):
    return np.exp(-((x - center)**2)/wid)

In [7]:
""" 2n-cis structure

         C17
     C33      C38
C29 (C45)    (C46)  C37
        (Pt1)
 C8 (N1)     (N2)  C16
     C7       C12
         C28
"""


""" 2n-trans structure
         C17
     C33      C38
C29 (C45)    (N1)  C37
        (Pt1)
 C8 (N2)     (C46) C16
     C7       C12
         C28
"""

'2n-cis//No_bias//'

In [151]:
def system_pair_pcoop_data(surf_idx, bias_idx, lookat1, lookat2, smear=False):
    #start_time = time.time()
    proj, nStates, nBands, nProj, nOrbsPerAtom, wk, k_points, E = cfunc.parse_data(app_dir(surf_idx, bias_idx) + "bandProjections",
                                                                                   app_dir(surf_idx, bias_idx) + "Gvectors",
                                                                                   app_dir(surf_idx, bias_idx) + "eigenvals")
    T_juk, P_uvjk, e_jk = cfunc.prepare_small_funcs(proj, E)
    orbs_dict = cfunc.orbs_idx_dict(app_dir(surf_idx, bias_idx) + "out", nOrbsPerAtom)
    S, R, mu = jfunc.get_vars(app_dir(surf_idx, bias_idx) + "out")
    nk, nj = np.shape(E)

    def onme(orb_idx):
        return orbital_labels[orb_idx]

    def savename(at1, orb1, at2, orb2, surf_idx, bias_idx):
        out = lsstr(at1) + "_" + onme(orb1) + "_" + lsstr(at2) + "_" + onme(orb2) + "_" + surf_labels[surf_idx] + "_" + str(bias_idx + 4)
        return out

    orbital_labels = []
    maxint = max(orbs_dict["Pt #1"]) + 2
    for i in range(maxint):
        orbital_labels.append("n/a")
    pt_orbs = ["6s", "6px", "6py", "6pz", "7px", "7py", "7pz", "5dxy", "5dyz", "5dxz", "5dx2y2", "5dz2"]
    ticker = 0
    for atom in orbs_dict.keys():
        if ("C " in atom) or ("N " in atom):
            orbital_labels[ticker] = "2s"
            orbital_labels[ticker + 1] = "2px"
            orbital_labels[ticker + 2] = "2py"
            orbital_labels[ticker + 3] = "2pz"
            ticker += 4
        else:
            for orb in pt_orbs:
                orbital_labels[ticker] = orb
                ticker += 1

    Emin = np.min(E)
    Emax = np.max(E)
    dE = 0.01
    Emax, Emin = cfunc.adjust_Ebounds(Emin, Emax, dE)
    Erange = np.arange(Emin, Emax, dE)
    orbs_sample1 = orbs_dict[lookat1]
    orbs_sample2 = orbs_dict[lookat2]

    pcoops_list = []
    mincoop = 0
    maxcoop = 0
    for u in orbs_sample1:
        pcoops_list.append([])
        for v in orbs_sample2:
            if smear:
                pcoops_list[-1].append(np.zeros((np.shape(Erange))))
                for ik in range(nk):
                    for ij in range(nj):
                        ejk = E[ik][ij]
                        height = P_uvjk(u, v, ij, ik)*wk[ik]
                        pcoops_list[-1][-1] += smear(Erange, ejk, 4*dE)*height
            else:
                pcoops_list[-1].append(list(np.zeros((np.shape(Erange)))))
                for ik in range(nk):
                    for ij in range(nj):
                        ejk = E[ik][ij]
                        idx, spill = lb_idx(Emin, dE, ejk)
                        full = P_uvjk(u, v, ij, ik)*wk[ik]
                        pcoops_list[-1][-1][idx] = full * (1-spill)
                        pcoops_list[-1][-1][idx] = full * (spill)
            mincoop = min(np.min(np.real(pcoops_list[-1][-1])), mincoop)
            maxcoop = max(np.max(np.real(pcoops_list[-1][-1])), maxcoop)
    titles_and_idcs_and_mu = []
    for i, u in enumerate(orbs_sample1):
        for j, v in enumerate(orbs_sample2):
            titles_and_idcs_and_mu.append([savename(lookat1, u, lookat2, v, surf_idx, bias_idx), [i, j], mu])
    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print('Data for surf ' + surf_labels[surf_idx] + ' at bias ' + bias_labels[bias_idx] + ' generated in ' + f"{elapsed_time}" + ' seconds.')
    """
    pcoops_list: 2d list
    Erange: 1d range
    titles_and_idcs_and_mu: [title, data i/j pair, mu]
    """
    return pcoops_list, Erange, titles_and_idcs_and_mu, mincoop, maxcoop

In [148]:
def save_plot_bias_range(surf_idx, lookat1, lookat2, dirname, smear=False):
    biases = [-4, -3, -2, -1, 0, 1, 2, 3, 4]
    data = []
    for bias_idx in biases:
        data.append(system_pair_pcoop_data(surf_idx, bias_idx, lookat1, lookat2, smear=smear))
    mincoop = 0
    maxcoop = 0
    for i in range(len(data)):
        mincoop = min(mincoop, data[i][3])
        maxcoop = max(maxcoop, data[i][4])
    for i in range(len(data)):
        for info in data[i][2]:
            title = info[0]
            mu = info[2]
            pcoop_data = data[i][0][info[1][0]][info[1][1]]
            Erange = data[i][1]
            plt.plot(Erange, np.real(pcoop_data))
            plt.ylim(mincoop, maxcoop)
            plt.title('pCOOP ' + title)
            plt.axvline(x=mu, linestyle="--", color="red")
            plt.savefig(dirname + title + '.png')
            plt.clf()

In [112]:
plot_sets = [
    [save_dir + "cisPt1-N1//", "Pt #1", "N #1", 0],
    [save_dir + "cisPt1-C45//", "Pt #1", "C #45", 0],
    [save_dir + "cisPt1-C46//", "Pt #1", "C #46", 0],
    [save_dir + "cisN1-C7//", "N #1", "C #7", 0],
    [save_dir + "cisN1-C8//", "N #1", "C #8", 0],
    [save_dir + "cisN2-C12//", "N #2", "C #12", 0],
    [save_dir + "cisN2-C16//", "N #2", "C #16", 0],
    [save_dir + "cisC45-C33//", "C #45", "C #33", 0],
    [save_dir + "cisC45-C29//", "C #45", "C #29", 0],
    [save_dir + "cisC46-C38//", "C #46", "C #38", 0],
    [save_dir + "cisC46-C37//", "C #46", "C #37", 0]
]

In [116]:
for ps in plot_sets:
    dirname = ps[0]
    os.mkdir(dirname)
    lookat1 = ps[1]
    lookat2 = ps[2]
    surf_idx = ps[3]
    save_plot_bias_range(surf_idx, lookat1, lookat2, dirname)

Elapsed time: 3.8734192848205566 seconds
Elapsed time: 3.209009885787964 seconds
Elapsed time: 3.55832576751709 seconds
Elapsed time: 3.332077980041504 seconds
Elapsed time: 3.521484375 seconds
Elapsed time: 3.6163694858551025 seconds
Elapsed time: 3.2416629791259766 seconds
Elapsed time: 3.5214169025421143 seconds
Elapsed time: 3.32857084274292 seconds


C:\Users\User\anaconda3\envs\Musgrave_scripts\lib\site-packages\matplotlib\cbook\__init__.py:1369: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
C:\Users\User\anaconda3\envs\Musgrave_scripts\lib\site-packages\matplotlib\transforms.py:2865: ComplexWarning: Casting complex values to real discards the imaginary part
  vmin, vmax = map(float, [vmin, vmax])


Elapsed time: 3.7706029415130615 seconds
Elapsed time: 3.354125499725342 seconds
Elapsed time: 3.476026773452759 seconds
Elapsed time: 3.6272571086883545 seconds
Elapsed time: 3.193098783493042 seconds
Elapsed time: 3.462643623352051 seconds
Elapsed time: 3.1956915855407715 seconds
Elapsed time: 3.593735456466675 seconds
Elapsed time: 3.224254608154297 seconds
Elapsed time: 3.712637186050415 seconds
Elapsed time: 3.496131181716919 seconds
Elapsed time: 3.2547807693481445 seconds
Elapsed time: 3.671992540359497 seconds
Elapsed time: 3.37090802192688 seconds
Elapsed time: 3.4968607425689697 seconds
Elapsed time: 3.0214412212371826 seconds
Elapsed time: 3.4168999195098877 seconds
Elapsed time: 3.2596242427825928 seconds
Elapsed time: 3.3816347122192383 seconds
Elapsed time: 3.1215765476226807 seconds
Elapsed time: 3.3943264484405518 seconds
Elapsed time: 3.09030818939209 seconds
Elapsed time: 3.3752284049987793 seconds
Elapsed time: 3.507101058959961 seconds
Elapsed time: 3.00123167037963

<Figure size 640x480 with 0 Axes>

In [132]:
def gen_plotset(lookat1, lookat2, surf_idx, save_dir):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    dirname = gen_dirname(lookat1, lookat2, surf_idx, save_dir)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    save_plot_bias_range(surf_idx, lookat1, lookat2, dirname)

In [135]:
def gen_plotsets(lookats1, lookats2, surf_idcs, save_dir):
    sets = len(lookats2)*len(lookats1)*len(surf_idcs)
    times = []
    for i, lookat1 in enumerate(lookats1):
        for j, lookat2 in enumerate(lookats2):
            for k, surf_idx in enumerate(surf_idcs):
                print('Interrogating system ' + str(len(times) + 1) + '/' + str(sets) + ' (' + lookat1 + ", " + lookat2 + ', ' + surf_labels[surf_idx] + ')')
                set_start = time.time()
                gen_plotset(lookat1, lookat2, surf_idx, save_dir)
                set_end = time.time()
                elapsed = set_end - set_start
                times.append(elapsed)
                t_per_set = np.sum(times)/len(times)
                t_remaining = (sets - len(times))*t_per_set
                print('Set ' + str(len(times)) + '/' + str(sets) + ' finished in ' + f"{elapsed} seconds.")
                print('Estimated ' + f"{t_remaining}" + ' seconds remaining.')

In [152]:
save_to = save_dir + 'exhaustive4dEsmear//'
atoms_look = ['C #17', 'C #33', 'C #38', 'C #29', 'C #45', 'C #46', 'C #37', 'Pt #1', 'C #8', 'N #1', 'N #2', 'C #16', 'C #7', 'C #28']
gen_plotsets(atoms_look, atoms_look, [0,1], save_to)

Interrogating system 1/392 (C #17, C #17, cis)
Set 1/392 finished in 48.53304839134216 seconds.
Estimated 18976.421921014786 seconds remaining.
Interrogating system 2/392 (C #17, C #17, trans)
Set 2/392 finished in 47.73355484008789 seconds.
Estimated 18771.98763012886 seconds remaining.
Interrogating system 3/392 (C #17, C #33, cis)
Set 3/392 finished in 47.661423683166504 seconds.
Estimated 18662.66748992602 seconds remaining.
Interrogating system 4/392 (C #17, C #33, trans)
Set 4/392 finished in 51.92372179031372 seconds.
Estimated 18997.619624376297 seconds remaining.
Interrogating system 5/392 (C #17, C #38, cis)
Set 5/392 finished in 47.98323369026184 seconds.
Estimated 18872.827637386323 seconds remaining.
Interrogating system 6/392 (C #17, C #38, trans)
Set 6/392 finished in 50.29684042930603 seconds.
Estimated 18922.480601708095 seconds remaining.
Interrogating system 7/392 (C #17, C #29, cis)
Set 7/392 finished in 49.65562963485718 seconds.
Estimated 18908.309885263443 second

<Figure size 640x480 with 0 Axes>

In [ ]:
def gen_dirname(lookat1, lookat2, surf_idx, save_dir):
    slabel = surf_labels[surf_idx]
    out_str = save_dir + slabel
    out_str += lookat1.split(' #')[0]
    out_str += lookat1.split(' #')[1]
    out_str += '-'
    out_str += lookat2.split(' #')[0]
    out_str += lookat2.split(' #')[1]
    out_str += '//'
    return out_str

In [ ]:
""" 2n-cis structure

         C17
     C33      C38
C29 (C45)    (C46)  C37
        (Pt1)
 C8 (N1)     (N2)  C16
     C7       C12
         C28
"""


""" 2n-trans structure
         C17
     C33      C38
C29 (C45)    (N1)  C37
        (Pt1)
 C8 (N2)     (C46) C16
     C7       C12
         C28
"""

In [ ]:
def save_plots(surf_idx, bias_idx, lookat1, lookat2, dir):
    start_time = time.time()
    proj, nStates, nBands, nProj, nOrbsPerAtom, wk, k_points, E = cfunc.parse_data(app_dir(surf_idx, bias_idx) + "bandProjections",
                                                                                   app_dir(surf_idx, bias_idx) + "Gvectors",
                                                                                   app_dir(surf_idx, bias_idx) + "eigenvals")
    T_juk, P_uvjk, e_jk = cfunc.prepare_small_funcs(proj, E)
    orbs_dict = cfunc.orbs_idx_dict(app_dir(surf_idx, bias_idx) + "out", nOrbsPerAtom)
    S, R, mu = jfunc.get_vars(app_dir(surf_idx, bias_idx) + "out")
    nk, nj = np.shape(E)

    def onme(orb_idx):
        return orbital_labels[orb_idx]

    def savename(at1, orb1, at2, orb2, surf_idx, bias_idx):
        out = lsstr(at1) + "_" + onme(orb1) + "_" + lsstr(at2) + "_" + onme(orb2) + "_" + surf_labels[surf_idx] + "_" + str(bias_idx)
        return out

    orbital_labels = []
    maxint = max(orbs_dict["Pt #1"]) + 2
    for i in range(maxint):
        orbital_labels.append("n/a")
    pt_orbs = ["6s", "6px", "6py", "6pz", "7px", "7py", "7pz", "5dxy", "5dyz", "5dxz", "5dx2y2", "5dz2"]
    ticker = 0
    for atom in orbs_dict.keys():
        if ("C " in atom) or ("N " in atom):
            orbital_labels[ticker] = "2s"
            orbital_labels[ticker + 1] = "2px"
            orbital_labels[ticker + 2] = "2py"
            orbital_labels[ticker + 3] = "2pz"
            ticker += 4
        else:
            for orb in pt_orbs:
                orbital_labels[ticker] = orb
                ticker += 1

    Emin = np.min(E)
    Emax = np.max(E)
    dE = 0.01
    Emax, Emin = cfunc.adjust_Ebounds(Emin, Emax, dE)
    Erange = np.arange(Emin, Emax, dE)
    orbs_sample1 = orbs_dict[lookat1]
    orbs_sample2 = orbs_dict[lookat2]

    pcoops_list = []
    mincoop = 0
    maxcoop = 0
    for u in orbs_sample1:
        pcoops_list.append([])
        for v in orbs_sample2:
            pcoops_list[-1].append(list(np.zeros((np.shape(Erange)))))
            for ik in range(nk):
                for ij in range(nj):
                    ejk = E[ik][ij]
                    idx, spill = lb_idx(Emin, dE, ejk)
                    full = P_uvjk(u, v, ij, ik)*wk[ik]
                    pcoops_list[-1][-1][idx] = full * (1-spill)
                    pcoops_list[-1][-1][idx] = full * (spill)
            mincoop = min(np.min(pcoops_list[-1][-1]), mincoop)
            maxcoop = max(np.max(pcoops_list[-1][-1]), maxcoop)
    for i, u in enumerate(orbs_sample1):
        for j, v in enumerate(orbs_sample2):
            plt.plot(Erange, pcoops_list[i][j])
            plt.ylim(mincoop, maxcoop)
            plt.title("pCOOP " + savename(lookat1, u, lookat2, v, surf_idx, bias_idx))
            plt.axvline(x=mu, linestyle="--", color="red")
            plt.savefig(save_dir + dir + savename(lookat1, u, lookat2, v, surf_idx, bias_idx) + ".png")
            plt.clf()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")

In [99]:
def save_plots(surf_idx, bias_idx, lookat1, lookat2, dir):
    start_time = time.time()
    proj, nStates, nBands, nProj, nOrbsPerAtom, wk, k_points, E = cfunc.parse_data(app_dir(surf_idx, bias_idx) + "bandProjections",
                                                                                   app_dir(surf_idx, bias_idx) + "Gvectors",
                                                                                   app_dir(surf_idx, bias_idx) + "eigenvals")
    T_juk, P_uvjk, e_jk = cfunc.prepare_small_funcs(proj, E)
    orbs_dict = cfunc.orbs_idx_dict(app_dir(surf_idx, bias_idx) + "out", nOrbsPerAtom)
    S, R, mu = jfunc.get_vars(app_dir(surf_idx, bias_idx) + "out")
    nk, nj = np.shape(E)

    def onme(orb_idx):
        return orbital_labels[orb_idx]

    def savename(at1, orb1, at2, orb2, surf_idx, bias_idx):
        out = lsstr(at1) + "_" + onme(orb1) + "_" + lsstr(at2) + "_" + onme(orb2) + "_" + surf_labels[surf_idx] + "_" + str(bias_idx)
        return out

    orbital_labels = []
    maxint = max(orbs_dict["Pt #1"]) + 2
    for i in range(maxint):
        orbital_labels.append("n/a")
    pt_orbs = ["6s", "6px", "6py", "6pz", "7px", "7py", "7pz", "5dxy", "5dyz", "5dxz", "5dx2y2", "5dz2"]
    ticker = 0
    for atom in orbs_dict.keys():
        if ("C " in atom) or ("N " in atom):
            orbital_labels[ticker] = "2s"
            orbital_labels[ticker + 1] = "2px"
            orbital_labels[ticker + 2] = "2py"
            orbital_labels[ticker + 3] = "2pz"
            ticker += 4
        else:
            for orb in pt_orbs:
                orbital_labels[ticker] = orb
                ticker += 1

    Emin = np.min(E)
    Emax = np.max(E)
    dE = 0.01
    Emax, Emin = cfunc.adjust_Ebounds(Emin, Emax, dE)
    Erange = np.arange(Emin, Emax, dE)
    orbs_sample1 = orbs_dict[lookat1]
    orbs_sample2 = orbs_dict[lookat2]

    pcoops_list = []
    mincoop = 0
    maxcoop = 0
    for u in orbs_sample1:
        pcoops_list.append([])
        for v in orbs_sample2:
            pcoops_list[-1].append(list(np.zeros((np.shape(Erange)))))
            for ik in range(nk):
                for ij in range(nj):
                    ejk = E[ik][ij]
                    idx, spill = lb_idx(Emin, dE, ejk)
                    full = P_uvjk(u, v, ij, ik)*wk[ik]
                    pcoops_list[-1][-1][idx] = full * (1-spill)
                    pcoops_list[-1][-1][idx] = full * (spill)
            mincoop = min(np.min(pcoops_list[-1][-1]), mincoop)
            maxcoop = max(np.max(pcoops_list[-1][-1]), maxcoop)
    for i, u in enumerate(orbs_sample1):
        for j, v in enumerate(orbs_sample2):
            plt.plot(Erange, pcoops_list[i][j])
            plt.ylim(mincoop, maxcoop)
            plt.title("pCOOP " + savename(lookat1, u, lookat2, v, surf_idx, bias_idx))
            plt.axvline(x=mu, linestyle="--", color="red")
            plt.savefig(save_dir + dir + savename(lookat1, u, lookat2, v, surf_idx, bias_idx) + ".png")
            plt.clf()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")